# Initial Exploration and Cleansing of cycleJourneys

In [4]:
journeys_path = '..\data\cycle_journeys\JourneysDataCombined.csv'
stations_path = '..\data\cycle_journeys\Station Lookup.csv'

In [22]:
import pandas as pd
import random
import networkx as nx
from matplotlib import pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

## Cleanse Stations
We will need to cleanse the station ids as certain files have corrupted schemas. 

But, this could turn-out to be quite a big part of the project! 

In [17]:
pd.read_csv(stations_path)

,Station ID,Station Name
0,268,"Belgrave Road, Victoria"
1,699,"Belford House , Haggerston"
2,10624,"Strata, Elephant & Castle"
3,519,Teviot Street: Poplar
4,3479,"South Kensington,0"
...,...,...
2907,3504,"Moor Street, Soho"
2908,112,"Stonecutter Street, Holborn"
2909,104,"Crosswall, Tower"
2910,1207,"Covent Garden,1"


## Sample Journeys

In [18]:
# We will sample just 1% of journeys, using skiprows
p = 0.01 

df_samp = pd.read_csv(
        journeys_path,
        header=0, 
        sep=',',
        skiprows=lambda i: i>0 and random.random() > p,
        error_bad_lines=False
)

In [20]:
df_samp.head()

,Rental Id,Duration,Bike Id,End Date,EndStation Id,Start Date,StartStation Id
0,50754250.0,360.0,12355.0,10/01/2016 00:16,573.0,10/01/2016 00:10,142.0
1,50754392.0,900.0,9108.0,10/01/2016 01:13,450.0,10/01/2016 00:58,278.0
2,50754793.0,300.0,7198.0,10/01/2016 03:26,487.0,10/01/2016 03:21,763.0
3,50754841.0,840.0,5251.0,10/01/2016 04:12,433.0,10/01/2016 03:58,462.0
4,50754949.0,1080.0,5195.0,10/01/2016 05:55,28.0,10/01/2016 05:37,226.0


In [21]:
df_samp['Start Date'].plot(kind='hist')

ModuleNotFoundError: No module named 'matplotlib'